In [1]:
using Rays
using Gtk

# Using `Gtk.jl`

### Define a function which produces a render for a given state

In [2]:
using LinearAlgebra: normalize!

# Define a camera
camera = Rays.Camera()
camera.screen_res .= [500, 500]

# Let the camera look towards the origin
from = Float32[2.0, 2.0, 2.0]
to = zeros(Float32, 3)
Rays.look_at!(camera, from, to)

# Define a cube at the origin
cube = Rays.Cube(zeros(Float32, 3), 1.0f0)

# Color data
data_variables = [:dim]
julia_green = Float32[0.22, 0.596, 0.149]
julia_purple = Float32[0.584, 0.345, 0.698]
julia_red = Float32[0.796, 0.235, 0.2]
julia_colors = hcat(julia_green, julia_purple, julia_red)
color = Rays.get_canvas(camera; color = true)

function get_render(dist, θ, ϕ)::Array{<:AbstractFloat, 3}

    # Adjust camera
    Rays.look_at!(camera, zeros(Float32,3), dist, θ, ϕ)

    # Get intersection data
    intersection_data = Rays.shape_view(camera, cube; data_variables)

    # Get grayscale image
    canvas_grayscale = Rays.cam_is_source(intersection_data.t);

    # Apply color to the image
    color .= 0.0
    Rays.add_color!(color, julia_colors, intersection_data.dim)
    canvas_color = Rays.apply_color(canvas_grayscale, color);

    return canvas_color
end 

get_render (generic function with 1 method)

### Define a callback function for the Gtk window

In [3]:
function callback(widget, event)
    if Char(event.keyval) == 'd'
        dist -= 0.1
        get_render(dist, θ, ϕ)
    end
    return nothing
end

callback (generic function with 1 method)

In [4]:
function rgb_to_pixbuf(img_rgb::AbstractArray{<:AbstractFloat,3})
    (h, w) = size(img_rgb)[2:3]
    data = Array{Gtk.RGB}(undef, h, w)
    for j in 1:h, i in 1:w
        data[j, i] = Gtk.RGB(
            convert(UInt8,round(255*img_rgb[1, i, j])),
            convert(UInt8,round(255*img_rgb[2, i, j])),
            convert(UInt8,round(255*img_rgb[3, i, j]))
        )
    end
    return GdkPixbuf(data=data, has_alpha=false)
end

rgb_to_pixbuf (generic function with 1 method)

In [5]:
win = GtkWindow("Cube view", camera.screen_res[2], camera.screen_res[1])
signal_connect(callback, win, "key_press_event")
θ = π/4
ϕ = 3π/8
dist = 10.0

canvas_color = get_render(dist, θ, ϕ)
pixbuf = rgb_to_pixbuf(canvas_color)
image_widget = Gtk.Image(pixbuf)
push!(win, image_widget)
showall(win)

GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Cube view", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=500, default-height=500, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint=F